# Creating outcomes tables and views for SMC cohort 

In [ ]:
import SAIL_python
import pandas as pd 
import numpy as np 
import glob
import os

In [ ]:
con = SAIL_python.SAILConnection()
con.connect()

# ALL TABLES

## WLGP outcomes tables - with version 

In [ ]:
#loading all the concept tables. 
#Note: if we want to create outcomes tables ONLY for a specific version of the look-up tables it is necessary to modify the name of the tables
q = """
    SELECT * FROM SYSCAT.TABLES WHERE TABSCHEMA = 'SAILW1377V' AND TABNAME LIKE 'CONC_READ_%' AND TYPE = 'T'
    """

tables = con.query(q)['TABNAME'] 
len(tables)

In [ ]:
pd.set_option('display.max_rows', None)
#WLGP extraction table 

q = """
    SELECT * FROM SYSCAT.TABLES WHERE TABSCHEMA = 'SAILW1377V' AND TABNAME LIKE 'CONC_READ_%' AND TYPE = 'T'
    """

tables = con.query(q)['TABNAME'] 

#sensitive concept -> if a look-up table has sensitive_flag = 1 for all the codes then do not create outcome table 

wlgp_extract = 'WP02_WLGP_EVENTS_EXT' #check the table name in Eclipse

count_wlgp = 0

for i in tables:
    table_name = i.removeprefix('CONC_READ_')
    
    count_wlgp = count_wlgp + 1

    q1 = """
         CREATE TABLE SAILW1377V.WP02_CONC_EXT_WLGP_%s (
            ALF_PE BIGINT, 
            PRAC_CD_PE BIGINT, 
            LOCAL_NUM_PE BIGINT, 
            EVENT_CD VARCHAR(40), 
            EVENT_CD_DESC VARCHAR(200), 
            EVENT_VAL DECIMAL(31,8), 
            EVENT_DT DATE, 
            EVENT_SEQ INTEGER
        )
        DISTRIBUTE BY HASH(ALF_PE)
        ; 
        """  % (table_name)

    table1 = con.query(q1)

    q2 = """
        INSERT INTO SAILW1377V.WP02_CONC_EXT_WLGP_%s --tablename
        SELECT 
            A.ALF_PE, A.PRAC_CD_PE, A.LOCAL_NUM_PE, 
            A.EVENT_CD, B.DESCRIPTION_SAIL, 
            A.EVENT_VAL, A.EVENT_DT, 
            RANK() OVER (PARTITION BY A.ALF_PE ORDER BY A.EVENT_DT) AS EVENT_SEQ
        FROM 
            SAILW1377V.%s A --wlgp_extract
        JOIN 
            SAILW1377V.CONC_READ_%s B --tablename 
        ON 
            A.EVENT_CD = B.READ_CODE 
        --JOIN 
            --SAILW1377V.WP02_COHORT_TABLE C 
        --ON 
            --A.ALF_PE = C.ALF_PE 
        WHERE 
            B.SENSITIVE_CODE = 0
        GROUP BY 
            A.ALF_PE, A.PRAC_CD_PE, A.LOCAL_NUM_PE, 
            A.EVENT_CD, B.DESCRIPTION_SAIL, 
            A.EVENT_VAL, A.EVENT_DT
        ORDER BY 
            A.ALF_PE, A.EVENT_DT, A.EVENT_CD; 
            """ % (table_name, wlgp_extract, table_name)

    insert = con.query(q2)

### WLGP view 

In [ ]:
#this line has to be modified with the latest version (in this case it was 2_2_3)

views = {i.replace('CONC_READ_', '').replace('_2_2_3', '') for i in tables}

In [ ]:
#creating view identical to more recent version tables 
#note: if a view wit hthe same name has already been created, this needs to be drop before creating the new ones.

for i in views:
    view_name = i.removeprefix('CONC_READ_')
    
    q_view = """
            CREATE VIEW SAILW1377V.WP02_CONC_EXT_WLGP_%s AS SELECT * FROM SAILW1377V.WP02_CONC_EXT_WLGP_%s_2_2_3
            """ %(i,i)

    view = con.query(q_view, echo_level = 0)

## PEDW outcomes tables

In [ ]:
#PEDW extraction table
q = """
    SELECT * FROM SYSCAT.TABLES WHERE TABSCHEMA = 'SAILW1377V' AND TABNAME LIKE 'CONC_ICD10_%' AND TYPE = 'T'
    """
tables = con.query(q)['TABNAME'] 
pedw_diag_ext = 'WP02_PEDW_DIAG_EXT'
pedw_epi_ext = 'WP02_PEDW_EPI_EXT'
pedw_spell_ext = 'WP02_PEDW_SPELL_EXT'
icd10_code = 'ICD10_CODES_AND_TITLES_AND_METADATA'

count_pedw = 0 

for i in tables:
    table_name = i.removeprefix('CONC_ICD10_')

    count_pedw = count_pedw + 1
    table_name = i.removeprefix('CONC_ICD10_')
    print('ADDING: ', table_name)

    q1 = """
         CREATE TABLE SAILW1377V.WP02_CONC_EXT_PEDW_%s ( 
            ALF_PE BIGINT,
            PROV_UNIT_CD VARCHAR(3),
            SPELL_NUM_PE BIGINT,
            ADMIS_DT DATE,
            DISCH_DT DATE,
            EPI_STR_DT DATE,
            EPI_END_DT DATE,
            EPI_NUM VARCHAR(2),
            DIAG_NUM INTEGER,
            DIAG_CD VARCHAR(20),
            DIAG_CD_123 VARCHAR(20),
            DIAG_CD_123_DESC VARCHAR(200),
            DIAG_CD_1234 VARCHAR(20),
            DIAG_CD_1234_DESC VARCHAR(200), 
            EVENT_SEQ INTEGER
        )
        DISTRIBUTE BY HASH(ALF_PE);
        ; 
        """  % (table_name)

    table1 = con.query(q1)

    q2 = """ 
        INSERT INTO SAILW1377V.WP02_CONC_EXT_PEDW_%s --tablename 
        SELECT DISTINCT
            ALF_PE, PROV_UNIT_CD, SPELL_NUM_PE,
            ADMIS_DT, DISCH_DT, EPI_STR_DT, EPI_END_DT,
            EPI_NUM, DIAG_NUM,
            DIAG_CD, 
            DIAG_CD_123, DIAG_CD_123_DESC, 
            DIAG_CD_1234, DIAG_CD_1234_DESC, 
            RANK() OVER (PARTITION BY ALF_PE ORDER BY ADMIS_DT, DISCH_DT, EPI_STR_DT, EPI_END_DT) AS EVENT_SEQ   
        FROM 
        (
            (
                SELECT       --Collection of EPI with ICD10 length = 3 
                    S.ALF_PE, D.PROV_UNIT_CD, D.SPELL_NUM_PE,
                    S.ADMIS_DT, S.DISCH_DT, E.EPI_STR_DT, E.EPI_END_DT,
                    D.EPI_NUM, D.DIAG_NUM,
                    D.DIAG_CD,
                    D.DIAG_CD_123, P.DESCRIPTION_SAIL_3DIGITS AS DIAG_CD_123_DESC, 
                    D.DIAG_CD_1234, P2.DESCRIPTION AS DIAG_CD_1234_DESC
                FROM 
                    SAILW1377V.%s D --pedw_diag
                JOIN
                    SAILW1377V.%s E --pedw_epi 
                ON
                    D.PROV_UNIT_CD = E.PROV_UNIT_CD
                AND
                    D.SPELL_NUM_PE = E.SPELL_NUM_PE
                AND
                    D.EPI_NUM = E.EPI_NUM
                JOIN
                    SAILW1377V.%s S --pedw_spell 
                ON
                    E.PROV_UNIT_CD = S.PROV_UNIT_CD
                AND
                    E.SPELL_NUM_PE = S.SPELL_NUM_PE	
                JOIN
                    SAILW1377V.CONC_ICD10_%s P  --tablename 
                ON 
                    D.DIAG_CD_123 = P.ICD10_CODE
                LEFT JOIN  
                    SAILUKHDV.%s P2 --icd10 table 
                ON 
                    D.DIAG_CD_1234 = P2.ALT_CODE 
                WHERE 
                    P.CODE_LENGTH = 3
                    AND 
                    P.SENSITIVE_CODE = 0 
            )
            UNION ALL
            (
                SELECT            -- Collection of EPI with ICD10 length = 4 
                    S.ALF_PE, D.PROV_UNIT_CD, D.SPELL_NUM_PE,
                    S.ADMIS_DT, S.DISCH_DT, E.EPI_STR_DT, E.EPI_END_DT,
                    D.EPI_NUM, D.DIAG_NUM,
                    D.DIAG_CD, 
                    D.DIAG_CD_123, P.DESCRIPTION_SAIL_3DIGITS  AS DIAG_CD_123_DESC, 
                    D.DIAG_CD_1234, P.DESCRIPTION_SAIL AS DIAG_CD_1234_DESC
                FROM 
                    SAILW1377V.%s D --pedw_doiag
                JOIN
                    SAILW1377V.%s E --pedw_epi
                ON
                    D.PROV_UNIT_CD = E.PROV_UNIT_CD
                AND
                    D.SPELL_NUM_PE = E.SPELL_NUM_PE
                AND
                    D.EPI_NUM = E.EPI_NUM
                JOIN
                    SAILW1377V.%s S --pedw_spell
                ON
                    E.PROV_UNIT_CD = S.PROV_UNIT_CD
                AND
                    E.SPELL_NUM_PE = S.SPELL_NUM_PE	
                JOIN
                    SAILW1377V.CONC_ICD10_%s P  --tablename 
                ON 
                    D.DIAG_CD_1234 = P.ICD10_CODE
                WHERE 
                    P.CODE_LENGTH = 4
                    AND 
                    P.SENSITIVE_CODE = 0 
            )
        )
        GROUP BY
            ALF_PE, PROV_UNIT_CD, SPELL_NUM_PE,
            ADMIS_DT, DISCH_DT, EPI_STR_DT, EPI_END_DT,
            EPI_NUM, DIAG_NUM,
            DIAG_CD,
            DIAG_CD_123, DIAG_CD_123_DESC, 
            DIAG_CD_1234, DIAG_CD_1234_DESC
        ORDER BY
            ALF_PE, 
            ADMIS_DT, DISCH_DT, EPI_STR_DT, EPI_END_DT,
            EPI_NUM, DIAG_NUM
        """ % (table_name, pedw_diag_ext, pedw_epi_ext, pedw_spell_ext, table_name, icd10_code, 
               pedw_diag_ext, pedw_epi_ext, pedw_spell_ext, table_name)

    insert = con.query(q2, echo_level = 0)

## VIEWS 
At the moment we need to insert the version manually for 2_2_3 and 2_2_4 -> to adapt to automatically detect the higher version number 

In [ ]:
q_tables_p = """
            SELECT * FROM SYSCAT.TABLES WHERE TABSCHEMA = 'SAILW1377V' AND TABNAME LIKE 'WP02_CONC_EXT_PEDW_%' AND type = 'T'
            """
tables_p = con.query(q_tables_p, echo_level = 0)['TABNAME']

In [ ]:
views_p = {i.replace('_2_2_3', '') for i in tables_p}

In [ ]:
#creating view identical to more recent version tables 
#note: if a view wit hthe same name has already been created, this needs to be drop before creating the new ones.
for i in views_p:
    
    q_view = """
            CREATE VIEW SAILW1377V.%s AS SELECT * FROM SAILW1377V.%s_2_2_3
            """ %(i,i)

    view = con.query(q_view, echo_level = 0)

In [ ]:
#checks 

q_tables_w = """
            SELECT * FROM SYSCAT.TABLES WHERE TABSCHEMA = 'SAILW1377V' AND TABNAME LIKE 'WP02_CONC_EXT_WLGP_%' AND type = 'V'
            """
tables_w = con.query(q_tables_w, echo_level = 0)
len(tables_w)

# SINGLE TABLE 

In this case we create a single look-up table specifying the concept name (e.g LONGCOVID)

## WLGP

In [ ]:
pd.set_option('display.max_rows', None)
#WLGP extraction table 

tables = ['LONGCOVID']

wlgp_extract = 'WP02_WLGP_EVENTS_EXT' #check the table name in Eclipse

for i in tables:
    table_name = i.removeprefix('CONC_READ_')
    print(table_name)
    print(i)
    
    q1 = """
         CREATE TABLE SAILW1377V.WP02_CONC_EXT_WLGP_%s_2_2_3 (
            ALF_PE BIGINT, 
            PRAC_CD_PE BIGINT, 
            LOCAL_NUM_PE BIGINT, 
            EVENT_CD VARCHAR(40), 
            EVENT_CD_DESC VARCHAR(200), 
            EVENT_VAL DECIMAL(31,8), 
            EVENT_DT DATE, 
            EVENT_SEQ INTEGER
        )
        DISTRIBUTE BY HASH(ALF_PE)
        ; 
        """  % (table_name)
    
    table1 = con.query(q1, echo_level = 0)
    
    q2 = """
        INSERT INTO SAILW1377V.WP02_CONC_EXT_WLGP_%s_2_2_3 --tablename
        SELECT 
            A.ALF_PE, A.PRAC_CD_PE, A.LOCAL_NUM_PE, 
            A.EVENT_CD, B.DESCRIPTION_SAIL, 
            A.EVENT_VAL, A.EVENT_DT, 
            RANK() OVER (PARTITION BY A.ALF_PE ORDER BY A.EVENT_DT) AS EVENT_SEQ
        FROM 
            SAILW1377V.%s A --wlgp_extract
        JOIN 
            SAILW1377V.CONC_READ_%s B --tablename 
        ON 
            A.EVENT_CD = B.READ_CODE 
        JOIN 
            SAILW1377V.WP02_COHORT_TABLE C 
        ON 
            A.ALF_PE = C.ALF_PE 
        WHERE 
            B.SENSITIVE_CODE = 0
        GROUP BY 
            A.ALF_PE, A.PRAC_CD_PE, A.LOCAL_NUM_PE, 
            A.EVENT_CD, B.DESCRIPTION_SAIL, 
            A.EVENT_VAL, A.EVENT_DT
        ORDER BY 
            A.ALF_PE, A.EVENT_DT, A.EVENT_CD; 
            """ % (table_name, wlgp_extract, table_name)
    
    insert = con.query(q2, echo_level = 0)
    

## PEDW 

In [ ]:
#PEDW extraction table
tables = ['LONGCOVID']

pedw_diag_ext = 'WP02_PEDW_DIAG_EXT'
pedw_epi_ext = 'WP02_PEDW_EPI_EXT'
pedw_spell_ext = 'WP02_PEDW_SPELL_EXT'
icd10_code = 'ICD10_CODES_AND_TITLES_AND_METADATA'

for i in tables:
    table_name = i.removeprefix('CONC_ICD10_')
    print(table_name)
    print(i)
    
    q1 = """
         CREATE TABLE SAILW1377V.WP02_CONC_EXT_PEDW_%s ( 
            ALF_PE BIGINT,
            PROV_UNIT_CD VARCHAR(3),
            SPELL_NUM_PE BIGINT,
            ADMIS_DT DATE,
            DISCH_DT DATE,
            EPI_STR_DT DATE,
            EPI_END_DT DATE,
            EPI_NUM VARCHAR(2),
            DIAG_NUM INTEGER,
            DIAG_CD VARCHAR(20),
            DIAG_CD_123 VARCHAR(20),
            DIAG_CD_123_DESC VARCHAR(200),
            DIAG_CD_1234 VARCHAR(20),
            DIAG_CD_1234_DESC VARCHAR(200), 
            EVENT_SEQ INTEGER
        )
        DISTRIBUTE BY HASH(ALF_PE);
        ; 
        """  % (table_name)
    
    table1 = con.query(q1)
    
    q2 = """ 
        INSERT INTO SAILW1377V.WP02_CONC_EXT_PEDW_%s --tablename 
        SELECT DISTINCT
            ALF_PE, PROV_UNIT_CD, SPELL_NUM_PE,
            ADMIS_DT, DISCH_DT, EPI_STR_DT, EPI_END_DT,
            EPI_NUM, DIAG_NUM,
            DIAG_CD, 
            DIAG_CD_123, DIAG_CD_123_DESC, 
            DIAG_CD_1234, DIAG_CD_1234_DESC, 
            RANK() OVER (PARTITION BY ALF_PE ORDER BY ADMIS_DT, DISCH_DT, EPI_STR_DT, EPI_END_DT) AS EVENT_SEQ   
        FROM 
        (
            (
                SELECT       --Collection of EPI with ICD10 length = 3 
                    S.ALF_PE, D.PROV_UNIT_CD, D.SPELL_NUM_PE,
                    S.ADMIS_DT, S.DISCH_DT, E.EPI_STR_DT, E.EPI_END_DT,
                    D.EPI_NUM, D.DIAG_NUM,
                    D.DIAG_CD,
                    D.DIAG_CD_123, P.DESCRIPTION_SAIL_3DIGITS AS DIAG_CD_123_DESC, 
                    D.DIAG_CD_1234, P2.DESCRIPTION AS DIAG_CD_1234_DESC
                FROM 
                    SAILW1377V.%s D --pedw_diag
                JOIN
                    SAILW1377V.%s E --pedw_epi 
                ON
                    D.PROV_UNIT_CD = E.PROV_UNIT_CD
                AND
                    D.SPELL_NUM_PE = E.SPELL_NUM_PE
                AND
                    D.EPI_NUM = E.EPI_NUM
                JOIN
                    SAILW1377V.%s S --pedw_spell 
                ON
                    E.PROV_UNIT_CD = S.PROV_UNIT_CD
                AND
                    E.SPELL_NUM_PE = S.SPELL_NUM_PE	
                JOIN
                    SAILW1377V.CONC_ICD10_%s P  --tablename 
                ON 
                    D.DIAG_CD_123 = P.ICD10_CODE
                LEFT JOIN  
                    SAILUKHDV.%s P2 --icd10 table 
                ON 
                    D.DIAG_CD_1234 = P2.ALT_CODE 
                WHERE 
                    P.CODE_LENGTH = 3
                    AND 
                    P.SENSITIVE_CODE = 0 
            )
            UNION ALL
            (
                SELECT            -- Collection of EPI with ICD10 length = 4 
                    S.ALF_PE, D.PROV_UNIT_CD, D.SPELL_NUM_PE,
                    S.ADMIS_DT, S.DISCH_DT, E.EPI_STR_DT, E.EPI_END_DT,
                    D.EPI_NUM, D.DIAG_NUM,
                    D.DIAG_CD, 
                    D.DIAG_CD_123, P.DESCRIPTION_SAIL_3DIGITS  AS DIAG_CD_123_DESC, 
                    D.DIAG_CD_1234, P.DESCRIPTION_SAIL AS DIAG_CD_1234_DESC
                FROM 
                    SAILW1377V.%s D --pedw_doiag
                JOIN
                    SAILW1377V.%s E --pedw_epi
                ON
                    D.PROV_UNIT_CD = E.PROV_UNIT_CD
                AND
                    D.SPELL_NUM_PE = E.SPELL_NUM_PE
                AND
                    D.EPI_NUM = E.EPI_NUM
                JOIN
                    SAILW1377V.%s S --pedw_spell
                ON
                    E.PROV_UNIT_CD = S.PROV_UNIT_CD
                AND
                    E.SPELL_NUM_PE = S.SPELL_NUM_PE	
                JOIN
                    SAILW1377V.CONC_ICD10_%s P  --tablename 
                ON 
                    D.DIAG_CD_1234 = P.ICD10_CODE
                WHERE 
                    P.CODE_LENGTH = 4
                    AND 
                    P.SENSITIVE_CODE = 0 
            )
        )
        GROUP BY
            ALF_PE, PROV_UNIT_CD, SPELL_NUM_PE,
            ADMIS_DT, DISCH_DT, EPI_STR_DT, EPI_END_DT,
            EPI_NUM, DIAG_NUM,
            DIAG_CD,
            DIAG_CD_123, DIAG_CD_123_DESC, 
            DIAG_CD_1234, DIAG_CD_1234_DESC
        ORDER BY
            ALF_PE, 
            ADMIS_DT, DISCH_DT, EPI_STR_DT, EPI_END_DT,
            EPI_NUM, DIAG_NUM
        """ % (table_name, pedw_diag_ext, pedw_epi_ext, pedw_spell_ext, table_name, icd10_code, 
               pedw_diag_ext, pedw_epi_ext, pedw_spell_ext, table_name)
    
    insert = con.query(q2, echo_level = 0)

# QA TABLES/VIEWS COUNTING

Checking number of outcomes tables vs look-up tables and views 

In [ ]:
q_read = """
    SELECT * FROM SYSCAT.TABLES WHERE TABSCHEMA = 'SAILW1377V' AND TABNAME LIKE 'CONC_READ_%' AND TYPE = 'T'
    """

tables_read = con.query(q_read)['TABNAME'] 

In [ ]:
#read check 

q_tables_n = """
            SELECT * FROM SYSCAT.TABLES WHERE TABSCHEMA = 'SAILW1377V' AND TABNAME LIKE 'WP02_CONC_EXT_WLGP_%' AND type = 'T'
            """
tables_n = con.query(q_tables_n, echo_level = 0)['TABNAME']

#check for diff 
t = len(tables_read) - len(tables_n)
print(t)

In [ ]:
q_icd10 = """
    SELECT * FROM SYSCAT.TABLES WHERE TABSCHEMA = 'SAILW1377V' AND TABNAME LIKE 'CONC_ICD10_%' AND TYPE = 'T'
    """

tables_icd10 = con.query(q_icd10)['TABNAME'] 

q_tables_n = """
            SELECT * FROM SYSCAT.TABLES WHERE TABSCHEMA = 'SAILW1377V' AND TABNAME LIKE 'WP02_CONC_EXT_PEDW_%' AND type = 'T'
            """
tables_n = con.query(q_tables_n, echo_level = 0)['TABNAME']

#check for diff 
t = len(tables_icd10) - len(tables_n)
print(t)